In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv("/home/shuyu/others/cs315/data/all.csv")
train = pd.read_csv("/home/shuyu/others/cs315/data/train.csv")
val = pd.read_csv("/home/shuyu/others/cs315/data/val.csv")
test = pd.read_csv("/home/shuyu/others/cs315/data/test.csv")


In [3]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer


temp = CountVectorizer(min_df=2, max_df=0.7, stop_words=stopwords.words('english'))
t1 = temp.fit_transform(df['Sentence'].values.astype('U')).toarray()
vocabulary = temp.get_feature_names()
vectorizer = CountVectorizer(min_df=2, max_df=0.7, stop_words=stopwords.words('english'), vocabulary=vocabulary)

X_train = vectorizer.fit_transform(train['Sentence'].values.astype('U')).toarray()
X_val = vectorizer.fit_transform(val['Sentence'].values.astype('U')).toarray()
X_test = vectorizer.fit_transform(test['Sentence'].values.astype('U')).toarray()

y_train = train['Label'].values
y_val = val['Label'].values
y_test = test['Label'].values

print('X_train: ', X_train.shape)
print('X_val: ', X_val.shape)
print('X_test: ', X_test.shape)

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support


def performence(y_test, y_pred):
    a = accuracy_score(y_test, y_pred)
    precision, recall, fscore, _ = \
        precision_recall_fscore_support(y_test, y_pred, average='weighted')
    print("Accuracy : {:.4f}".format(a))
    print("Precision : {:.4f}".format(precision))
    print("Recall : {:.4f}".format(recall))
    print("F-Score : {:.4f}".format(fscore))


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


set_seed(108)

X_train = torch.from_numpy(X_train).cuda()
X_val = torch.from_numpy(X_val).cuda()
X_test = torch.from_numpy(X_test).cuda()

y_train = torch.from_numpy(y_train).cuda()
y_val = torch.from_numpy(y_val).cuda()
y_test = torch.from_numpy(y_test)


X_train:  (26979, 10319)
X_val:  (3372, 10319)
X_test:  (3374, 10319)


In [4]:
class LSTM(nn.Module):
    def __init__(self, in_dim=10319, dropout=0.0):
        super(LSTM, self).__init__()
        self.dropout = dropout

        self.lstm = nn.LSTM(in_dim, 512, batch_first=True)

        self.mlp = nn.Sequential(
            nn.Linear(in_features=512, out_features=64),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(self.dropout),
            nn.Linear(in_features=64, out_features=2),
        )
                
    def forward(self, x):
        x = torch.unsqueeze(x, 1)
        ou, _ = self.lstm(x)
        x = torch.squeeze(ou, 1)
        return self.mlp(x)


In [5]:
in_dim = X_train.shape[1]
dropout = 0.01
lr = 6e-6

lstm = LSTM(in_dim, dropout).cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(lstm.parameters(), lr=lr,
                      momentum=0.9, weight_decay=5e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

epoch = 30
batch = 100

max_loss = 1e6

In [6]:
for i in range(0, epoch):
    lstm.train()
    for j in range(0, batch):
        X_in = X_train[j:j + batch]
        y_true = y_train[j:j + batch]
        pred = lstm(X_in.float())
        loss = criterion(pred, y_true)
        loss.backward()
        optimizer.step()
    lstm.eval()
    pred = lstm(X_val.float())
    loss = criterion(pred, y_val)
    lo = loss.item()
#     if lo < max_loss:
#         max_loss = lo
#         dic = mlp.state_dict()
    print('Iter:{:d}, Loss:{:.2f}'.format(i, lo))


Iter:0, Loss:0.67
Iter:1, Loss:0.64
Iter:2, Loss:0.64
Iter:3, Loss:0.67
Iter:4, Loss:0.70
Iter:5, Loss:0.69
Iter:6, Loss:0.65
Iter:7, Loss:0.61
Iter:8, Loss:0.61
Iter:9, Loss:0.63
Iter:10, Loss:0.59
Iter:11, Loss:0.52
Iter:12, Loss:0.52
Iter:13, Loss:0.54
Iter:14, Loss:0.47
Iter:15, Loss:0.34
Iter:16, Loss:0.34
Iter:17, Loss:0.35
Iter:18, Loss:0.30
Iter:19, Loss:0.45
Iter:20, Loss:0.51
Iter:21, Loss:0.42
Iter:22, Loss:0.25
Iter:23, Loss:0.36
Iter:24, Loss:0.22
Iter:25, Loss:0.45
Iter:26, Loss:0.75
Iter:27, Loss:0.94
Iter:28, Loss:1.01
Iter:29, Loss:0.96
